<a href="https://colab.research.google.com/github/mertyyanik/Natural-Language-Processing-Real-or-Fake/blob/master/News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Doğal Dil İşleme - Gerçek ya da Sahte?
Doğal Dil işleme (Natural Language Processing), basit hali ile yapay zeka ve dil biliminin birleşmesi ile ortaya çıkmış bir kavramdır. Bu kavram herkesin günlük konuşma dilinin bilgisayara öğretilmesi ve bazı çalışmaların bilgisayara yaptırılmasına sebep olmuştur. Günümüze yakın tarihlere baktığımız zaman bilgisayarların güçlenmesini takip eden makine öğrenmesi modelleri daha da gelişmiş ve bir çok alanda kullanılmaya başlanmıştır. Bu alanlardan biriside doğal dil işlemedir. Doğal dil işlemeyi makine öğrenmesi modelleri ile birleştirdiğimiz zaman ortaya bir cümlenin kime ait olduğunu ya da bir cümleyi söyleyen kişinin nasıl bir ruh haline sahip olduğunu anlayabilir noktaya gelmiş bulunmaktayız.

Bizimde amacımız buna benzer bir çalışma yapmaktır. Elimizde haber sitelerinin yazmış olduğu haber başlıkları ve bu başlıkların gerçek ya da sahte olduğunu gösteren bir veri kümesi bulunmaktadır. Derin öğrenme yöntemlerini kullanarak bu veri kümesini bilgisayara öğretecek ve gelecekte yazılmış bir haber başlığının gerçek mi yoksa sahte mi olduğunu anlamaya çalışacağız.

Bunun için Google'ın geliştirmiş olduğu Colab ortamını kullanıyoruz. Bu ortamın avantajı derin öğrenme yöntemlerinde uygulamaları eğitirken **TPU**'ları kullanıyor olmasıdır. TPU'lar, GPU veya CPU'ya göre 15 kat daha hızlıdır. Bu da bizi büyük bir oranda zaman maliyetinden kurtarmaktadır.

Öncelikle elimizde ki veri kümelerini Colab ortamına aktarmamız gerekiyor. Bunu yapmadan önce ortamın Google Drive ile olan bağlantısını onaylamamız lazım. Aşağıda ki kod bloğunu çalıştırdığımız zaman ekranda bir link belirmektedir bu linke tıklayarak karşımıza çıkan kodu output kısmına yapıştırarak Google Drive ile olan bağlantıyı sağlamış oluyoruz.

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
 
from pydrive.drive import GoogleDrive
 
from google.colab import auth
 
from oauth2client.client import GoogleCredentials
 
auth.authenticate_user()
 
gauth = GoogleAuth()
 
gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)

Bundan sonraki aşama veri kümelerini Google Drive'dan indirmektir. Burada ki önemli nokta dosya numarasının (File ID) elde edilmesidir. Bu numaraya  Google Drive'dan üzerinden veri kümesinin paylaş butonuna basarak ulaşabilirsiniz.

In [0]:
#Test.csv Dosyası
from google.colab import files

file_id = '1Zx7P0t9lQLP4aj4YOUaZDxNRrSkZOTk4'

downloaded = drive.CreateFile({'id': file_id})

downloaded.GetContentFile('test.csv')

#Test.txt Dosyası
from google.colab import files
 
file_id = '1S2-N2HSIgtZpSzPmAEZWmVxV2YlOzCzs'
 
downloaded = drive.CreateFile({'id': file_id})
 
downloaded.GetContentFile('test.txt')

#clean_real-Train.txt Dosyası
from google.colab import files
 
file_id = '1pnsf7izWzNYlA41xcWrCXQ5SSFe5JCUF'
 
downloaded = drive.CreateFile({'id': file_id})
 
downloaded.GetContentFile('clean_real-Train.txt')

#clean_fake-Train.txt Dosyası
from google.colab import files
 
file_id = '1Mc1m6S6WCJuRkMBWM9_LZGHSzJIw2-e0'
 
downloaded = drive.CreateFile({'id': file_id})
 
downloaded.GetContentFile('clean_fake-Train.txt')


Bu noktadan sonra artık kendi Python kodlamanızı istediğiniz gibi yazabiliriz.

Öncelikle gerekli kütüphaneleri yükleyelim.

In [0]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score
import time

In [0]:

# Reading Data
input1Data = open('clean_real-Train.txt', 'r+')
input2Data = open('clean_fake-Train.txt', 'r+')
testData = open('test.txt')

testDataForComparison = pd.read_csv('test.csv').iloc[:, 1:]

input1List = []
input2List = []
testList = []

read = input1Data.readline()
read2 = input2Data.readline()
read3 = testData.readline()
rowCount = 0
while read != "":
    input1List.append(read)
    read = input1Data.readline()
    
while read2 != "":
    input2List.append(read2)
    read2 = input2Data.readline()

while read3 != "":
    testList.append(read3)
    read3 = testData.readline()
 

In [0]:
# Real and Fake data combined.    
liste = ['real' for i in range(len(input1List))]
realInput = np.column_stack((input1List, liste))


liste = ['fake' for i in range(len(input2List))]
fakeInput = np.column_stack((input2List, liste))

mainInput = realInput.tolist() + fakeInput.tolist()
random.shuffle(mainInput)

sentenceInput = []
targetInput = []
targetInputFloat = []
for i in range(len(mainInput)):
    sentenceInput.append(mainInput[i][0])
    
for i in range(len(mainInput)):
    targetInput.append(mainInput[i][1])
    
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
targetInputFloat = lb.fit_transform(targetInput)

In [0]:
# Stop Words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stemmer
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

# Regular Expressions
import re

sentenceList = []
for i in range(len(sentenceInput)):
    # Regular Expressions
    comment = re.sub('[^a-zA-Z]', ' ', sentenceInput[i])
    # Upper Case Problem
    comment = comment.lower()
    # Splitting
    comment = comment.split()
    # Stemming and Stop Words
    comment = [ps.stem(kelime) for kelime in comment if not kelime in set(stopwords.words('english'))]
    
    comment = ' '.join(comment)
    sentenceList.append(comment)
    


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# Vectorized All Data
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sentenceList)

result = pd.DataFrame(data=X.toarray())

In [0]:
# PCA - Principle Component Analysis
from sklearn.decomposition import PCA
pca = PCA().fit(result)

plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Bileşen Sayısı')
plt.ylabel('Varyans (%)') #for each component
plt.title('Veri Kümesi Varyans Yüzdeleri')
plt.show()

pca = PCA(n_components=2000)
result = pca.fit_transform(result)

In [0]:
# Test Data
testSentenceList = []
for i in range(len(testList)):
    # Regular Expressions
    comment = re.sub('[^a-zA-Z]', ' ', testList[i])
    # Upper Case Problem
    comment = comment.lower()
    # Splitting
    comment = comment.split()
    # Stemming and Stop Words
    comment = [ps.stem(kelime) for kelime in comment if not kelime in set(stopwords.words('english'))]
    
    comment = ' '.join(comment)
    testSentenceList.append(comment)

X = vectorizer.transform(testSentenceList)

testResult = pd.DataFrame(data=X.toarray())
# PCA for Test
testResult = pca.transform(testResult)


In [0]:
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import TensorBoard
import datetime

time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

kerasboard = TensorBoard(log_dir="/tmp/tensorboard/{}".format(time))

#result = result.reshape((2777, 1, 2000))
#testResult = testResult.reshape((489, 1, 2000))

classifier = Sequential()

classifier.add(Dense(3000, kernel_initializer="uniform", activation='sigmoid', input_dim=2000)) #1 -> Giriş Katmanı
#classifier.add(LSTM(1500, input_shape=(1, 2000), activation='sigmoid',return_sequences=False))
classifier.add(Dropout(0.50))


classifier.add(Dense(3000,kernel_initializer='uniform' ,activation='sigmoid')) #2 -> Saklı Katman
classifier.add(Dropout(0.50))


classifier.add(Dense(1, kernel_initializer="uniform", activation='sigmoid')) #3 -> Çıkış Katmanı

classifier.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

classifier.fit(result, targetInputFloat, epochs=6, callbacks=[kerasboard])

prediction = classifier.predict(testResult)

prediction = (prediction > 0.5)

from sklearn.metrics import accuracy_score

acScoreNN = accuracy_score(lb.fit_transform(testDataForComparison), prediction)
print("Accuracy : " + str(acScoreNN))

probaNN = classifier.predict_proba(testResult)
print("NN AUC " + str(metrics.roc_auc_score(lb.fit_transform(testDataForComparison), probaNN)))

print("tensorboard --logdir="+kerasboard.log_dir)

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard.notebook

#Activate Tensorboard
%tensorboard --logdir=/tmp/tensorboard/2019_03_17_03_30_47

In [0]:
#Saving the best model with Pickle (Neural %84.04)
import pickle
pickle.dump(classifier, open("NeuralNews", 'wb'))

loading = pickle.load(open("NeuralNews", 'rb'))
predictionPickleNeural = loading.predict(testResult)

predictionPickleNeural = (predictionPickleNeural > 0.5)

acScorePickleNeural = accuracy_score(lb.fit_transform(testDataForComparison), predictionPickleNeural)
print("Accuracy Pickle Neural : " + str(acScorePickleNeural))